# NAR Metadata Fixer (Robust Version)

地方競馬 (NAR) データのメタデータ（天気、馬場、コース、回りなど）の欠損を補完するための専用ノートブックです。
**特徴:**
- database_nar.csv を対象にします
- 50件ごとの自動保存機能
- nar.netkeiba.com へのスクレイピング

In [1]:
# 1. Google Driveのマウントと設定
from google.colab import drive
drive.mount('/content/drive')

import os
import sys

# プロジェクトパスの設定
PROJECT_PATH = '/content/drive/MyDrive/dai-keiba'

if not os.path.exists(PROJECT_PATH):
    print(f"Error: Path {PROJECT_PATH} does not exist.")
else:
    print(f"Project path found: {PROJECT_PATH}")
    os.chdir(PROJECT_PATH)
    sys.path.append(PROJECT_PATH)
    sys.path.append(os.path.join(PROJECT_PATH, 'scraper'))

Mounted at /content/drive
Project path found: /content/drive/MyDrive/dai-keiba


In [2]:
# 2. ライブラリのインポート
import pandas as pd
import re
import time
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

# TQDM (プログレスバー)
try:
    from tqdm.auto import tqdm
except ImportError:
    !pip install tqdm
    from tqdm.auto import tqdm

In [3]:
# 3. ロジック定義 (NAR対応版)

def local_get_nar_race_metadata(rid):
    """
    Fetches Turn, Weather, Course, Condition from Netkeiba NAR Page.
    """
    # NAR URL structure
    url = f"https://nar.netkeiba.com/race/result.html?race_id={rid}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        resp.encoding = 'EUC-JP' # Netkeiba usually EUC-JP
        if resp.status_code != 200: return None
        soup = BeautifulSoup(resp.text, 'html.parser')

        data = {'race_id': rid}

        # Check RaceData01 (Common for both JRA and NAR pages on Netkeiba)
        rd1 = soup.select_one(".RaceData01")
        if rd1:
            txt = rd1.text.strip()

            # Weather
            if "天候:晴" in txt: data["weather"] = "晴"
            elif "天候:曇" in txt: data["weather"] = "曇"
            elif "天候:小雨" in txt: data["weather"] = "小雨"
            elif "天候:雨" in txt: data["weather"] = "雨"
            elif "天候:雪" in txt: data["weather"] = "雪"

            # Condition
            if "馬場:良" in txt: data["condition"] = "良"
            elif "馬場:稍" in txt: data["condition"] = "稍重"
            elif "馬場:重" in txt: data["condition"] = "重"
            elif "馬場:不良" in txt: data["condition"] = "不良"

            # Course
            match = re.search(r'(芝|ダ|障)(\d+)m', txt)
            if match:
                ctype_raw = match.group(1)
                if ctype_raw == "芝": data["course_type"] = "芝"
                elif ctype_raw == "ダ": data["course_type"] = "ダート"
                elif ctype_raw == "障": data["course_type"] = "障害"
                data["distance"] = match.group(2)

            # Turn (NAR often has specific turns like '右' or '左')
            if "右" in txt: data["turn"] = "右"
            elif "左" in txt: data["turn"] = "左"
            elif "直線" in txt: data["turn"] = "直"

        return data
    except Exception as e:
        # print(f"Error {rid}: {e}")
        return None

def fill_missing_nar_metadata_robust():
    csv_path = os.path.join(PROJECT_PATH, 'database_nar.csv')
    if not os.path.exists(csv_path):
        print(f'Error: {csv_path} not found.')
        return

    print(f'Reading {csv_path}...')
    df = pd.read_csv(csv_path, low_memory=False, dtype={'race_id': str})

    # Ensure columns exist
    print('Checking Metadata (Course, Distance, Weather, Turn)...')
    meta_cols = ['コースタイプ', '距離', '天候', '馬場状態', '回り']
    for c in meta_cols:
        if c not in df.columns: df[c] = None

    # Identify missing rows
    # Check specifically for missing Course Type as a proxy for missing metadata
    missing_meta_mask = (df['コースタイプ'].isna()) | (df['コースタイプ'] == '') | (df['回り'].isna())
    if 'race_id' in df.columns:
         rids_missing = df.loc[missing_meta_mask, 'race_id'].astype(str).unique()
    else:
         rids_missing = []

    print(f'Found {len(rids_missing)} races with missing metadata in NAR DB.')

    if len(rids_missing) > 0:
        BATCH_SIZE = 50
        batch_buffer = {}
        processed_count = 0

        print("Starting batch processing... (Saving every 50 races)")

        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = {executor.submit(local_get_nar_race_metadata, rid): rid for rid in rids_missing}

            for future in tqdm(as_completed(futures), total=len(rids_missing), desc="Fetching NAR Metadata"):
                try:
                    data = future.result()
                    if data and data.get('course_type'):
                        batch_buffer[str(data['race_id'])] = data
                except: pass

                processed_count += 1

                # BATCH SAVE
                if len(batch_buffer) >= BATCH_SIZE:
                    print(f'  Saving batch... ({processed_count}/{len(rids_missing)})')

                    df['race_id_str'] = df['race_id'].astype(str)
                    mask = df['race_id_str'].isin(batch_buffer.keys())

                    # Create maps
                    c_map = {rid: d.get('course_type') for rid, d in batch_buffer.items()}
                    d_map = {rid: d.get('distance') for rid, d in batch_buffer.items()}
                    w_map = {rid: d.get('weather') for rid, d in batch_buffer.items()}
                    cond_map = {rid: d.get('condition') for rid, d in batch_buffer.items()}
                    t_map = {rid: d.get('turn') for rid, d in batch_buffer.items()}

                    # Update DataFrame
                    df.loc[mask, 'コースタイプ'] = df.loc[mask, 'race_id_str'].map(c_map).fillna(df.loc[mask, 'コースタイプ'])
                    df.loc[mask, '距離'] = df.loc[mask, 'race_id_str'].map(d_map).fillna(df.loc[mask, '距離'])
                    df.loc[mask, '天候'] = df.loc[mask, 'race_id_str'].map(w_map).fillna(df.loc[mask, '天候'])
                    df.loc[mask, '馬場状態'] = df.loc[mask, 'race_id_str'].map(cond_map).fillna(df.loc[mask, '馬場状態'])
                    df.loc[mask, '回り'] = df.loc[mask, 'race_id_str'].map(t_map).fillna(df.loc[mask, '回り'])

                    # Save to CSV
                    temp_df = df.drop(columns=['race_id_str'], errors='ignore')
                    temp_df.to_csv(csv_path, index=False, encoding='utf-8-sig')

                    # Clear buffer
                    batch_buffer = {}

            # FINAL SAVE
            if batch_buffer:
                print('  Saving final batch...')
                df['race_id_str'] = df['race_id'].astype(str)
                mask = df['race_id_str'].isin(batch_buffer.keys())

                c_map = {rid: d.get('course_type') for rid, d in batch_buffer.items()}
                d_map = {rid: d.get('distance') for rid, d in batch_buffer.items()}
                w_map = {rid: d.get('weather') for rid, d in batch_buffer.items()}
                cond_map = {rid: d.get('condition') for rid, d in batch_buffer.items()}
                t_map = {rid: d.get('turn') for rid, d in batch_buffer.items()}

                df.loc[mask, 'コースタイプ'] = df.loc[mask, 'race_id_str'].map(c_map).fillna(df.loc[mask, 'コースタイプ'])
                df.loc[mask, '距離'] = df.loc[mask, 'race_id_str'].map(d_map).fillna(df.loc[mask, '距離'])
                df.loc[mask, '天候'] = df.loc[mask, 'race_id_str'].map(w_map).fillna(df.loc[mask, '天候'])
                df.loc[mask, '馬場状態'] = df.loc[mask, 'race_id_str'].map(cond_map).fillna(df.loc[mask, '馬場状態'])
                df.loc[mask, '回り'] = df.loc[mask, 'race_id_str'].map(t_map).fillna(df.loc[mask, '回り'])

                df.drop(columns=['race_id_str'], inplace=True, errors='ignore')
                df.to_csv(csv_path, index=False, encoding='utf-8-sig')

        print('Done filling metadata for NAR.')
    else:
        print('All metadata present for NAR.')

In [4]:
# 4. 実行
fill_missing_nar_metadata_robust()

Reading /content/drive/MyDrive/dai-keiba/database_nar.csv...
Checking Metadata (Course, Distance, Weather, Turn)...
Found 420 races with missing metadata in NAR DB.
Starting batch processing... (Saving every 50 races)


Fetching NAR Metadata:   0%|          | 0/420 [00:00<?, ?it/s]

Done filling metadata for NAR.
